In [2]:
%load_ext autoreload

In [66]:
%autoreload
import numpy as np
from dctkit.mesh import util, simplex
from dctkit.math.opt import optctrl as oc
import dctkit.dec.cochain as C
import dctkit as dt
import pygmsh
import pyvista as pv
from pyvista import themes

In [67]:
dt.config()
pv.set_jupyter_backend('trame')
pv.global_theme = themes.ParaViewTheme()

In [68]:
lc = 0.2
with pygmsh.geo.Geometry() as geom:
        poly = geom.add_polygon([[0.0, 0.0], [1.0, 0.0], [1.0, 1.0], [0.0, 1.0]], lc)

        top, volume, lat = geom.extrude(poly, [0, 0, 1.0])
        
        geom.add_physical(poly, label="bottom")
        geom.add_physical(top, label="top")
        geom.add_physical(volume, label="volume")
        geom.add_physical(lat, label="lat")
        mesh = geom.generate_mesh()

#pv.plot(mesh)

In [69]:
bottom_faces_ids = mesh.cell_sets_dict['bottom']['triangle']

In [70]:
node_coords = mesh.points
num_nodes = node_coords.shape[0]
tet_node_tags = mesh.cells[2].data
print("number of nodes = ", num_nodes)
print("number of tets = ", tet_node_tags.shape[0])

number of nodes =  237
number of tets =  741


In [71]:
spx = simplex.SimplicialComplex(tet_node_tags, node_coords, is_well_centered=True)
spx.get_circumcenters()
spx.get_primal_volumes()
spx.get_dual_volumes()
spx.get_hodge_star()
# S.get_dual_edge_vectors()
# S.get_flat_weights()

In [72]:
# boundary conditions
# bottom_nodes = spx.S[2][bottom_faces_ids]
# bottom_nodes = np.unique(bottom_nodes.flatten())
bottom_nodes = np.argwhere(spx.node_coord[:,2]<1e-6).flatten()
top_nodes = np.argwhere(abs(spx.node_coord[:,2]-1.)<1e-6).flatten()
values = np.zeros(len(bottom_nodes)+len(top_nodes), dtype=dt.float_dtype)
boundary_values = (np.hstack((bottom_nodes,top_nodes)), values)
boundary_values

(array([  0,   1,   2,   3,   8,   9,  10,  11,  12,  13,  14,  15,  16,
         17,  18,  19,  20,  21,  22,  23,  56,  57,  58,  59,  60,  61,
         62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,
         75,  76,  77,  78,  79,   4,   5,   6,   7,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39, 178,
        179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191,
        192, 193, 194, 195, 196, 197, 198, 199, 200, 201]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]))

In [73]:
from functools import partial
from dctkit.apps import poisson as p

obj = partial(p.energy_poisson, S=spx)
k = 1.
f_vec = np.ones(num_nodes, dtype=dt.float_dtype)
gamma = 1000.
args = {'f': f_vec, 'k': k, 'boundary_values': boundary_values,
        'gamma': gamma}

u_0 = np.zeros(num_nodes, dt.float_dtype)

In [74]:
prb = oc.OptimizationProblem(dim=num_nodes, state_dim=num_nodes, objfun=obj)
prb.set_obj_args(args)
u = prb.run(u_0, ftol_abs=1e-8, ftol_rel=1e-8)
prb.last_opt_result

1

In [75]:
mesh.point_data['u'] = u
pv.plot(mesh)

Widget(value="<iframe src='http://localhost:38941/index.html?ui=P_0x7f69a8143cd0_7&reconnect=auto' style='widt…